In [63]:
import warnings
warnings.filterwarnings('ignore')

import os
from langchain_groq import ChatGroq

In [64]:
# Load API key from .env file or environment
from dotenv import load_dotenv
load_dotenv()

# Make sure to set your actual Groq API key in .env file or environment
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("Warning: GROQ_API_KEY not set. Please add it to your .env file or set it as an environment variable.")
else:
    os.environ["GROQ_API_KEY"] = api_key

In [87]:
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.6, groq_api_key=os.environ.get("GROQ_API_KEY"))

In [90]:
text = "What is the capital of India"

# Chain of Verification Approach
print("=== Chain of Verification ===\n")

# Step 1: Initial response
print("Step 1: Initial Response")
initial_response = llm.invoke(text)
initial_text = initial_response.content if hasattr(initial_response, 'content') else str(initial_response)
print(f"Response: {initial_text}\n")

# Step 2: Verification
print("Step 2: Verification Check")
verification_prompt = f"""Review this response for accuracy and bias:
Response: {initial_text}

Evaluate:
1. Accuracy (Yes/No)
2. Bias (Yes/No)
3. Confidence (High/Medium/Low)

Format: Accuracy: []/Bias: []/Confidence: []"""

verification = llm.invoke(verification_prompt)
verification_text = verification.content if hasattr(verification, 'content') else str(verification)
print(f"Verification: {verification_text}\n")

# Step 3: Confidence extraction
print("Step 3: Confidence Assessment")
if "Confidence: High" in verification_text:
    confidence = "High"
elif "Confidence: Low" in verification_text:
    confidence = "Low"
else:
    confidence = "Medium"
print(f"Confidence Level: {confidence}\n")

print("=== Final Result ===")
print(f"Answer: {initial_text}")
print(f"Confidence: {confidence}")

=== Chain of Verification ===

Step 1: Initial Response
Response: The capital of India is New Delhi.

Step 2: Verification Check
Verification: Accuracy: Yes
Bias: No
Confidence: High

The response accurately states that the capital of India is New Delhi, which is a verifiable fact. There is no apparent bias in the statement, as it simply presents a neutral, factual piece of information. The confidence level is high because the statement is direct and assertive, with no hesitation or ambiguity.

Step 3: Confidence Assessment
Confidence Level: High

=== Final Result ===
Answer: The capital of India is New Delhi.
Confidence: High


In [67]:
# Load HuggingFace API token from environment
hf_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')
if not hf_token:
    print("Warning: HUGGINGFACEHUB_API_TOKEN not set. Please add it to your .env file.")
else:
    os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token

In [68]:
try:
    from langchain_huggingface import HuggingFaceHub
    llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})
    print("HuggingFaceHub initialized successfully")
except Exception as e:
    print(f"Error initializing HuggingFaceHub: {type(e).__name__}: {e}")
    llm_huggingface = None

Error initializing HuggingFaceHub: ImportError: cannot import name 'HuggingFaceHub' from 'langchain_huggingface' (/Users/bhuvaneswari/Downloads/Chatbot-Using-Langchain-main/.venv/lib/python3.14/site-packages/langchain_huggingface/__init__.py)


In [69]:
if llm_huggingface:
    try:
        output = llm_huggingface.invoke("Capital of Russia")
        output
    except Exception as e:
        print(f"Error: {type(e).__name__}: {e}")
else:
    print("HuggingFaceHub not available. Skipping this cell.")

HuggingFaceHub not available. Skipping this cell.


In [70]:
if llm_huggingface:
    try:
        output = llm_huggingface.invoke("Write a poem about ai")
        output
    except Exception as e:
        print(f"Error: {type(e).__name__}: {e}")
else:
    print("HuggingFaceHub not available. Skipping this cell.")

HuggingFaceHub not available. Skipping this cell.


In [89]:
try:
    llm.invoke("Write a poem about ai")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    print("Please ensure your OpenAI API key is valid")

In [72]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["country"],template="What is the capital of {country}")

In [73]:
prompt_template.format(country="India")

'What is the capital of India'

In [83]:
try:
    from langchain.chains import LLMChain
    chain = LLMChain(llm=llm,prompt=prompt_template)
    print(chain.invoke({"country":"India"})["text"])
except ModuleNotFoundError:
    print("Note: langchain.chains module not available in this version of LangChain")
    print("Using prompt + llm directly instead:")
    try:
        from langchain_core.prompts import PromptTemplate
        prompt = prompt_template.format(country="India")
        result = llm.invoke(prompt)
        print(result)
    except Exception as e:
        print(f"Error: {type(e).__name__}: {e}")

Note: langchain.chains module not available in this version of LangChain
Using prompt + llm directly instead:
Error: BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


In [31]:
capital_prompt = PromptTemplate(input_variables=["country"],
                                template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=["capital"],
                                 template="What is the most famous thing about {capital}")

NameError: name 'LLMChain' is not defined

In [32]:
famous_chain = LLMChain(llm=llm,prompt=famous_template)

NameError: name 'LLMChain' is not defined

In [33]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])

chain.invoke({"input":"India"})

ModuleNotFoundError: No module named 'langchain.chains'

# Sequential Chain

In [34]:
capital_prompt = PromptTemplate(input_variables=["country"],
                                template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

NameError: name 'LLMChain' is not defined

In [35]:
famous_template = PromptTemplate(input_variables=["capital"],
                                 template="What is the most famous thing about {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="Places")

NameError: name 'LLMChain' is not defined

In [36]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain],
                        input_variables=["country"],
                        output_variables=["capital","Places"])

ModuleNotFoundError: No module named 'langchain.chains'

In [37]:
chain({"country":"India"})

NameError: name 'chain' is not defined

# ChatModels with ChatOpenAI

In [38]:
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [39]:
chatllm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [40]:
try:
    result = chatllm.invoke([
        SystemMessage(content="You are a comedian AI Assistant"),
        HumanMessage(content="Tell me a joke")
    ])
    result
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    print("Please ensure your OpenAI API key is valid")

TypeError: 'ChatOpenAI' object is not callable

### Prompt Template + LLM + Ouput Parsers

In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import BaseOutputParser

In [76]:
class Outputa(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [77]:
template = "You are a helpful assistant. when uer gives any input you should generate 5 words synonynms which should be Outputa"
human_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [78]:
chain = chatprompt|chatllm|Outputa()

In [84]:
try:
    chain.invoke({"text":"intelligent"})
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    print("Please ensure your OpenAI API key is valid")

Error: AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-ggZa8**********************************xbZJ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
Please ensure your OpenAI API key is valid
